In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import spreg
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.formula.api as smf
from spreg import OLS

In [61]:
well_filter = 1
current_dir = os.getcwd()
path = os.path.join(current_dir, '../../data/clean/aligned_data', f"merged_dataset_{well_filter}.csv")
df = pd.read_csv(path)

In [62]:
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)

In [63]:
df["year"] = df["date"].dt.year
df["nitrate"] = np.log1p(df["nitrate"])

In [65]:
df_1 = df.drop(columns=['landuse code', 'soil region', 'bro-id', 'lon', 'lat'])
print(df_1.columns)
df_1 = df_1.dropna()
len(df_1)

Index(['nitrate', 'geometry', 'date', 'population', 'groundwater depth',
       'elevation', 'precipitation', 'temperature', 'n deposition',
       'mainsoilclassification_1', 'organicmattercontent_1', 'density_1',
       'acidity_1', 'year'],
      dtype='object')


132

In [14]:
predefined_categories = {
    "soil region": None,
    "landuse code": None,
    "mainsoilclassification_1": None
}
for col, _ in predefined_categories.items():
    if col in df_1.columns:
        cats = sorted(df_1[col].dropna().unique().tolist())
        df_1[col] = pd.Categorical(df_1[col], categories=cats)

In [15]:
categorical_cols = df.select_dtypes(include="category").columns.tolist()
numerical_cols = df.select_dtypes(include=["float64", "int64", "int32"]).columns.tolist()

In [16]:
df_cat = pd.get_dummies(df[categorical_cols], drop_first=True)
# df_cat = pd.get_dummies(df[categorical_cols], prefix='sc')
numerical_cols = df.select_dtypes(include=["float64", "int64", "int32"]).columns.tolist()

df_num = df[numerical_cols]

In [17]:
df_all = pd.concat([df_num, df_cat], axis=1).astype(float)

In [18]:
train_years = [2012, 2013, 2014, 2015, 2016, 2017]
test_years  = [2018, 2019, 2020]

In [19]:
train_df = df_all[df_all["year"].isin(train_years)].copy()
test_df  = df_all[df_all["year"].isin(test_years)].copy()

In [20]:
vars_to_keep = train_df.columns[train_df.nunique() > 1]
train_df = train_df[vars_to_keep]

In [21]:
common_cols = vars_to_keep.intersection(test_df.columns)
test_df = test_df[common_cols]

In [23]:
y_train = train_df["nitrate"]
X_train = train_df.drop(columns=["nitrate", "year"], errors='ignore').copy()
print(len(X_train))

y_test = test_df["nitrate"]
X_test = test_df.drop(columns=["nitrate", "year"], errors='ignore').copy()
print(len(X_test))

84
48


In [24]:
numerical_cols = [
 'population',
 'groundwater depth',
 'elevation',
 'precipitation',
 'temperature',
 'n deposition',
 'organicmattercontent_1',
 'density_1',
 'acidity_1']

In [25]:
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train[numerical_cols])
X_test_num_scaled  = scaler.transform(X_test[numerical_cols])

In [26]:
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns=X_train[numerical_cols].columns, index=X_train[numerical_cols].index)
X_test_num_scaled  = pd.DataFrame(X_test_num_scaled,  columns=X_test[numerical_cols].columns, index=X_test[numerical_cols].index)

In [27]:
X_train_num_scaled

,population,groundwater depth,elevation,precipitation,temperature,n deposition,organicmattercontent_1,density_1,acidity_1
0,-0.148449,1.469004,0.017696,-0.365835,-1.446659,0.186508,5.394886,-4.309724,0.286364
1,-0.100116,1.469004,0.496345,-0.365835,-1.446659,0.965924,-0.937001,0.587270,-0.069122
3,-0.293445,-0.065068,-0.367215,-0.504541,-1.457449,0.722084,-0.661702,0.871154,-1.372571
4,-0.220947,-0.970522,-0.404636,-0.504541,-1.457449,1.275078,-0.386402,-0.625150,1.115831
6,-0.293445,-1.542684,-0.928101,-0.500066,-1.453958,-1.498597,1.815993,-2.482223,0.641850
...,...,...,...,...,...,...,...,...,...
114,-0.148449,-1.086806,-0.404636,-0.151064,1.439568,0.273594,-0.386402,-0.625150,1.115831
115,-0.293445,-1.026085,-0.816965,0.229259,-0.048449,-1.002210,0.714795,-0.211153,0.404859
116,-0.293445,0.319448,-0.093457,1.737129,-0.363780,-0.880290,0.026547,0.279730,0.286364
120,-0.124282,-0.150241,0.009693,1.070445,-0.407347,1.375226,-0.661702,0.871154,-1.372571


In [28]:
X_train_cat = X_train.drop(columns=numerical_cols)
X_test_cat = X_test.drop(columns=numerical_cols)

In [29]:
X_train_df = pd.concat([X_train_num_scaled, X_train_cat], axis=1)
X_test_df = pd.concat([X_test_num_scaled, X_test_cat], axis=1)

## Statsmodel OLS

In [30]:
X_train_sm = sm.add_constant(X_train_df)
X_test_sm  = sm.add_constant(X_test_df)

In [31]:
model = sm.OLS(y_train, X_train_sm).fit()

In [32]:
print("Train shape:", X_train_sm.shape)
print("Test shape:", X_test_sm.shape)
print("\nOLS Regression Results:\n")
print(model.summary())

Train shape: (84, 16)
Test shape: (48, 16)

OLS Regression Results:

                            OLS Regression Results                            
Dep. Variable:                nitrate   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.710
Method:                 Least Squares   F-statistic:                     14.55
Date:                Fri, 06 Jun 2025   Prob (F-statistic):           1.03e-15
Time:                        13:26:46   Log-Likelihood:                -60.110
No. Observations:                  84   AIC:                             152.2
Df Residuals:                      68   BIC:                             191.1
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

In [33]:
y_pred_log = model.predict(X_test_sm)

In [34]:
y_pred_original = np.expm1(y_pred_log)
y_test_original = np.expm1(y_test)

In [35]:
r2_log = r2_score(y_test, y_pred_log)

In [36]:
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
mae = mean_absolute_error(y_test_original, y_pred_original)
r2_orig = r2_score(y_test_original, y_pred_original)

In [37]:
metrics = {
    "Metric": [
        "RMSE (original scale)", 
        "MAE (original scale)", 
        "R² (original scale)", 
        "R² (log scale)"
    ],
    "Value": [rmse, mae, r2_orig, r2_log]
}

metrics_df = pd.DataFrame(metrics)
metrics_df

,Metric,Value
0,RMSE (original scale),1.978890
1,MAE (original scale),1.264080
2,R² (original scale),0.750577
3,R² (log scale),0.676143


## Spreg OLS

In [38]:
X_train_const = X_train_df.copy()
X_test_const = X_test_df.copy()

In [39]:
X_train_ols = X_train_const.values
y_train_ols = y_train.values.reshape(-1, 1)

X_test_ols = X_test_const.values
y_test_ols = y_test.values.reshape(-1, 1)

In [40]:
X_train_df

,population,groundwater depth,elevation,precipitation,temperature,n deposition,organicmattercontent_1,density_1,acidity_1,mainsoilclassification_1_Kalkloze zandgronden,mainsoilclassification_1_Moerige gronden,mainsoilclassification_1_Podzolgronden,mainsoilclassification_1_Rivierkleigronden,mainsoilclassification_1_Veengronden,mainsoilclassification_1_Zeekleigronden
0,-0.148449,1.469004,0.017696,-0.365835,-1.446659,0.186508,5.394886,-4.309724,0.286364,0.0,1.0,0.0,0.0,0.0,0.0
1,-0.100116,1.469004,0.496345,-0.365835,-1.446659,0.965924,-0.937001,0.587270,-0.069122,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.293445,-0.065068,-0.367215,-0.504541,-1.457449,0.722084,-0.661702,0.871154,-1.372571,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.220947,-0.970522,-0.404636,-0.504541,-1.457449,1.275078,-0.386402,-0.625150,1.115831,0.0,0.0,0.0,1.0,0.0,0.0
6,-0.293445,-1.542684,-0.928101,-0.500066,-1.453958,-1.498597,1.815993,-2.482223,0.641850,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,-0.148449,-1.086806,-0.404636,-0.151064,1.439568,0.273594,-0.386402,-0.625150,1.115831,0.0,0.0,0.0,1.0,0.0,0.0
115,-0.293445,-1.026085,-0.816965,0.229259,-0.048449,-1.002210,0.714795,-0.211153,0.404859,0.0,0.0,0.0,0.0,1.0,0.0
116,-0.293445,0.319448,-0.093457,1.737129,-0.363780,-0.880290,0.026547,0.279730,0.286364,1.0,0.0,0.0,0.0,0.0,0.0
120,-0.124282,-0.150241,0.009693,1.070445,-0.407347,1.375226,-0.661702,0.871154,-1.372571,1.0,0.0,0.0,0.0,0.0,0.0


In [44]:
model_1 = spreg.OLS(y_train_ols, X_train_ols, name_y='nitrate', name_x=feature_names)
print(model_1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :     nitrate                Number of Observations:          84
Mean dependent var  :      0.9442                Number of Variables   :          16
S.D. dependent var  :      1.0215                Degrees of Freedom    :          68
R-squared           :      0.7624
Adjusted R-squared  :      0.7100
Sum squared residual:     20.5761                F-statistic           :     14.5483
Sigma-square        :       0.303                Prob(F-statistic)     :   1.027e-15
S.E. of regression  :       0.550                Log likelihood        :     -60.110
Sigma-square ML     :       0.245                Akaike info criterion :     152.220
S.E of regression ML:      0.4949                Schwarz criterion     :     191.113

------------------------------------------------------------

## Spatial fixed effects

In [57]:
df_2 = df.drop(columns=['landuse code', 'bro-id', 'lon', 'lat'])
df_2 = df_2.dropna()
len(df_2)

132

In [58]:
df_2.columns

Index(['nitrate', 'geometry', 'date', 'soil region', 'population',
       'groundwater depth', 'elevation', 'precipitation', 'temperature',
       'n deposition', 'mainsoilclassification_1', 'organicmattercontent_1',
       'density_1', 'acidity_1', 'year'],
      dtype='object')

In [48]:
all_columns = X_train_df.columns
all_columns

Index(['population', 'groundwater depth', 'elevation', 'precipitation',
       'temperature', 'n deposition', 'organicmattercontent_1', 'density_1',
       'acidity_1', 'mainsoilclassification_1_Kalkloze zandgronden',
       'mainsoilclassification_1_Moerige gronden',
       'mainsoilclassification_1_Podzolgronden',
       'mainsoilclassification_1_Rivierkleigronden',
       'mainsoilclassification_1_Veengronden',
       'mainsoilclassification_1_Zeekleigronden'],
      dtype='object')

In [46]:
formula = "nitrate ~ " + " + ".join([f"Q('{col}')" for col in all_columns]) + " + C(Q('soil region')) - 1"

In [47]:
model_2 = smf.ols(formula, data=X_train_sm).fit()

PatsyError: Error evaluating factor: NameError: no data named 'soil region' found
    nitrate ~ Q('population') + Q('groundwater depth') + Q('elevation') + Q('precipitation') + Q('temperature') + Q('n deposition') + Q('organicmattercontent_1') + Q('density_1') + Q('acidity_1') + Q('mainsoilclassification_1_Kalkloze zandgronden') + Q('mainsoilclassification_1_Moerige gronden') + Q('mainsoilclassification_1_Podzolgronden') + Q('mainsoilclassification_1_Rivierkleigronden') + Q('mainsoilclassification_1_Veengronden') + Q('mainsoilclassification_1_Zeekleigronden') + C(Q('soil region')) - 1
                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
print(model.summary())